In [ ]:
import sys
sys.path.append("..")

from include.RandomHelper import check_data_state
check_data_state()

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {return false;}

In the first part of the task, the data sets that were provided had to be examined and reduced in such a way that only a certain phase space remained - containing events that have a signal-like signature - a decay into four leptons via a Z boson pair, where one Z boson is virtual.

As a final step, a very rough estimate of the significance was made ($Z \approx s/\sqrt{b}$) and it was discussed why this is unsuitable.

In this part of the task two methods are presented how the significance of a measurement can be determined. The first method (local $p_0$ value) was used in the discovery of the Higgs boson - a simplified version of this method is presented in this section. The second method determines the significance using the signal probability.

In both methods, we will not consider the question of systematic uncertainty, since the addition of higher-dimensional fits has to be made, which are not necessarily more complicated, but much more time-consuming.

You can continue to use the obtained data sets from part one here.

# Parameterization of the background
------------------

To avoid fluctuations it is advantageous to express the background distribution of the MC simulations in the first step by a probability density.

The difference between a fit and a parameterization should again be clarified: The application of an fit always occurs when a certain theory previously established for it is to be confirmed or falsified by the measured data. The data contained for the fitting have uncertainty, which depends on the experiment and the measurement methods, but is fixed mainly by the measurement of the data.

In the case of a parameterization, the aim is to express the existing measurement, which in this example is available in the form of a histogram, by a different form. In this case it should be a continuous function - more precisely: a continuous probability density function (pdf) in a certain interval.

Thus, the main difference between an adaptation and a parameterization is clear: While in the first case a dependence given by the theory is present, in the second case an attempt is made to find a parameterization that describes the data in another form as accurately as possible.

For this reason - and especially in this case - it is not possible to find an exact function for it.Nevertheless to be able to make a selection and to justify it, the procedure after $\chi^2$ can be used: For a $\chi^2$ fit the uncertainties on the data are decisive whether a suitable model can be chosen for the data ($\frac{\chi^2}{n_{\mathrm{df}}}$) or not. Conversely, the uncertainties on the data can be scaled until a model fits. The question, which model parameterizes the data better, can now be answered with the scaling factor.

However, not only the smallest scaling factor is the decisive point, since this can always be realized by a polynomial of a very high degree. The so-called overfitting should be avoided, because only the data behaviour should be parameterized and not the fluctuation observed in this measurement. The choice should therefore be made for a function which still contains a sufficiently small number of parameters, but already keeps the scaling factor for the uncertainty as small as possible.

<div class="alert alert-info">

Use a suitable model to parameterize the background. Also make sure that the model you choose is the most suitable one.
    
</div>

The implementation of linear combinations of the legendre polynomials has already been carried out and summarized in the class `LLC`. Independent functions can also be used (see the numpy documentation).

The use of linear combinations of legendre polynomials offers the advantage over normal polynomials that the legendre polynomials form an orthogonal function system and behave robustly in case of an fit - unlike normal polynomials. The correlation of the uncertainties on the individual parameters is reduced due to the orthogonality, whereas with normal polynomials there is almost always a strong correlation between the individual parameters.

In [ ]:
import matplotlib.pyplot as plt
from include.parameterization.LegendrePoly import LegendreLinearCombination as LLC

In [ ]:
llc_background = LLC(x_mean=0.0) # centering around x_mean

x_ = np.linspace(-1, 1, 1000)
plt.plot(x_, llc_background.grade_0(x_, a=1))
plt.plot(x_, llc_background.grade_1(x_, a=1, b=1))
plt.plot(x_, llc_background.grade_2(x_, a=1, b=1, c=1))

In preparation, the number of bin numbers and the considered interval should be selected:

In [ ]:
bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/"  
llc_background = LLC(x_mean=(999.1 + 999.2)/2.)

Variant:

 1. <ins>The data from the histogram is raw output.</ins> 
<div class="alert alert-info">
     
    * Calculate the uncertainties on the individual entries of the unscaled histogram
    * Scale the histograms of the individual channels and combine them (and the calculated combined uncertainties) in a common histogram.

    (The individual parts can be accessed via the 'data' attribute, where it is a nested 'dict'. The first level contains the three individual channels. Each of the channels then contains the unscaled histogram entries, the correction factor and the data used for histogram creation).

</div>

The uncertainties thus calculated are fundamentally different from the $\sqrt{N}$ variant - they describe the scaled fluctuation of the data points.

In [ ]:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="background")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_ZZ_4el_2012"]

In [ ]:
# parametrization
import kafe2 as K2
# code

2. <ins>The data from the histogram is a summarized output</ins> 
    <div class="alert alert-info">
     
    Use the already combined values for a histogram fit.


</div>

In [ ]:
from include.fits.McFitInit import McFitInit

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir, info=info)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

In [ ]:
# parametrization

The actual fit is done by using the histogram fit implemented in 'kafe2'. An example of a histogram fit can be found [here](https://kafe2.readthedocs.io/en/latest/parts/user_guide.html#example-5-histogram-fit)

It is advantageous to create a function or class to avoid lengthy repetitions.

# Selection of the appropriate signal MC
-----------------------------------------

The goal of this section is to narrow down the possible signal MC simulations and to decide on a single one and continue working with it.

## Preliminary parameterization of signal MC
------------------

Different signal MC simulations are available. These were roughly parameterized by a Gaussian distribution $\mathcal{G} \left( m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G} \right)$. The aim is to determine the dependence of the width on the invariant mass and to describe it using a suitable model. The resolution of the detector's transversal pulse is the biggest influence.

The results of these parameterizations are summarized in a csv file:

The results of these Gaussian parameterizations are summarized in a Pandas Dataframe.

In [ ]:
import pandas as pd
df_ = pd.read_csv("../data/for_long_analysis/other_mc_gauss_param/gauss_params.csv")
df_
sigma, sigma_err, mu, mu_err = tuple(df_[["sigma", "sigma_err", "mu", "mu_err"]].values.T)

<div class="alert alert-info">

Parameterize $\sigma_{\mathrm{G}}(m_{4\ell})$ using the existing data.
  
</div>

In [ ]:
def sigma_model(x, *args):
    """
    Model to for sigma parameterization
    
    """
    # code
    
    return x

# code

This variant - the continuous parameterization of the signal MC does not happen in the correct analysis and is used here mainly for time reasons. Normally, a separate MC simulation is created for each Higgs mass, with which all further steps are performed. However, creating such a simulation is extremely time-consuming. A linear interpolation of a Gaussian-like parameterization, however, already leads to a sufficient result, that we want to achieve in the context of this section: The justification of the choice of a certain signal MC simulation based on the measurement - as well as an estimation of the statistical significance (more about this in the next sections).

## Likelihood-ratio test
-----------------

The likelihood ratio test is explicitly used for the selection between the different simulations.
For this purpose, the null hypothesis $H_0$, which only corresponds to the background, and the counter hypothesis $H_1$, which corresponds to the background and a signal, are compared with each other.

The extended unbinned likelihood of the $H_0$ hypothesis is based on the parameterization of the background $f_{\mathrm{U}}(x|\theta_{\mathrm{U}})$ where $f_{\mathrm{U}}$ is the previously determined model from the background parameterization with the optimal parameters $\theta_{\mathrm{U}}$.

The distribution used for the alternative hypothesis can be described by $$f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) = \mu f_{\mathrm{S}}(m_{4\ell}|\theta_{\mathrm{S}}) + f_{\mathrm{U}}(m_{4\ell}|\theta_{\mathrm{U}}) \, , \\ \int_{\Omega} f_{\mu\mathrm{S+B}}(m_{4\ell}|\mu) \mathrm{d}m_{4\ell} = 1 $$ $f_{\mathrm{S}}(x|\theta_{\mathrm{S}})$ is a Gaussian distribution $\mathcal{G}(m_{4\ell}, m_{\mathrm{H}}, \sigma_{\mathrm{G}}(m_{\mathrm{H}} ))$ with fixed values for the mass and width. For the estimator $\mu$ the limitation applies: $$\mu \geq 0 \, .$$ This restriction leads to a negligible distortion and is introduced for physical considerations, since in the distribution of invariant masses a excess of events is expected if a Higgs boson is present.

Thus, the quantity to be considered - the ratio of the likelihoods of both hypotheses - can be formed. For numerical stability, the negative logarithm of this is formed. The factor two is necessary for a later used asymptotic form.

$$ q_0 = -2 \ln \left( \frac{\mathcal{L}(x|\mu = \hat{\mu})}{\mathcal{L}(x|\mu = 0)} \right) \, .$$ 
$\hat{\mu}$ here is the ideal estimator for a fixed mass and width of the signal hypothesis.

The test statistic $q_0$ used here is a simplified version of the test statistic used at the LHC for this analysis. It corresponds to the test statistics used at the LEP and has the disadvantage of the bias, that is already mentioned. However, the advantage of the test statistic is that it is easier to implement and the calculation takes less time because no Toy Monte Carlo simulations are necessary. Likewise, even for a small number of events the analytical form can be used, which the test statistic takes in the borderline case of many events.

Remember: The extended likelihood function is defined as follows: $$ \mathcal{L}(x|\mu) = \frac{\nu(\mu)^N}{N!}\mathrm{e}^{-\nu(\mu)}\prod_i^N f_{\mu\mathrm{S+B}}(x_i|\mu) $$ and differs in the scaling factor from the normal likelihood function. Due to this scaling factor the number of measurements (since it is a counting experiment) plays a certain role.

In [ ]:
lower_bound, upper_bound = hist_range
mass_array = np.linspace(lower_bound, upper_bound, 100)
sigma_model_arguments = tuple([])
sigma_array = sigma_model(mass_array, *sigma_model_arguments)

<div class="alert alert-info">

Implement the likelihood ratio test. For numerical integration, you can use the ['scipy.integrate.quad'](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html) method. For minimizing the likelihood function you can use ['iminuit'](https://nbviewer.jupyter.org/github/scikit-hep/iminuit/blob/master/tutorial/basic_tutorial.ipynb)
).

</div>

In [ ]:
# measurement
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 9999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/" 
ru_dir = "../data/for_long_analysis/ru_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
data = raw_data.data
data

# scale factors for the mc simulations
b_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
_, _, background_hist, _ = b_mc_d.variables_for_hist_fit
scale_b_mc = np.sum(background_hist)

s_mc_d = McFitInit(bins=bins, hist_range=hist_range, tag="signal", mc_dir=mc_dir)
_, _, signal_hist, _ = s_mc_d.variables_for_hist_fit
scale_s_mc = np.sum(signal_hist)

scale_s_mc, scale_b_mc

In [ ]:
# Iminuit Example:
from iminuit import Minuit

def extended_likelihood(mu=0.0):
    # FUnction that need to be minimized
    #code
    return mu

m = Minuit(extended_likelihood, 
           mu=0.5, error_mu=0.1, limit_mu=(0.0, 3.0),
           errordef=0.5)
m.migrad()
best_mu = m.values["mu"]


q0 = np.zeros(len(mass_array))

#code

The asymptotic form of the probability density of $q_0$ when no signal is expected is described by
$$ f(q_0|\hat{\mu}=0) \rightarrow \frac{1}{2}\left( \delta (q_0) + \chi^2_{1} \right) \, .$$
from this the $p_0$ value can be calculated according to:
$$ p_0 = \int\limits_{q_{0_{\mathrm{obs}}}}^{\infty} \mathrm{d}q_0 f(q_0|\hat{\mu}=0) \, . $$
using this, the significance can be estimated as follows: $$Z = \Phi^{-1}(1-p_0) = \sqrt{q_{0_{\mathrm{obs}}}} \, .$$ However, in this case it should only be considered as a first estimation because the parameterization of the input signal is not accurate.
Likewise, the systematics are not considered at this point, only the statistical uncertainty is included in the calculation of $p_0$ or $Z$.

In 2012, the particle excess and the assignment of this excess to a Higgs boson which then led to its discovery was carried out using this method. 

Furthermore, another method for determining significance is presented, which - although with slightly different assumptions - leads to a similar significance as in the last part of the task and provides a good possibility for a cross-check.

It is useful to save the temporary result:

In [ ]:
q0, p0 = np.zeros(len(sigma_array)), np.zeros(len(sigma_array))
df = pd.DataFrame(data=np.array([mass_array, sigma_array, q0, p0, np.sqrt(q0)]).T,
                  columns=["mass_array", "gauss_sigma", "q0", "p0", "sqrt_q0"])

<div class="alert alert-info">

Visualize the results. For the significance levels the 'get_p0_sigma_lines' from 'PlotHelper' can be used.
</div>

In [ ]:
import matplotlib.pyplot as plt
from include.stattest.PlotHelper import PlotHelper

In [ ]:
fig, ax = plt.subplots(1,1)
PlotHelper.get_p0_sigma_lines(x_=mass_array, 
                             ax_obj_=ax, 
                             max_sigma_=4)
ax.plot(df.mass_array, df.p0+1)  # <- change this line accordingly
plt.show()

# Examination of the final distributions
-----------------------

This section is a repetition and is available in the first section - but offers a good possibility to visualize the initial situation again - with the only difference that there is now a reason why exactly the 125 GeV Signal MC Simulation was used.


Other quantities can still be displayed (although it is questionable whether 'z1_index', 'z2_index', as well as 'z1_tag' or 'z2_tag' are quantities that require detailed visual consideration):

In [ ]:
from include.processing.ApplyHelper import ProcessHelper
print(ProcessHelper.print_possible_variables("../data/for_long_analysis/mc_aftH/MC_2012_H_to_ZZ_to_4L_2el2mu_aftH.csv"))

<div class="alert alert-info">

Examine the distribution of the four lepton invariant masses and explain the $p_0$ value, which depends on the four lepton masses, in relation to the measured events. Why is there no change in the $p_0$ value for larger masses (when the mass interval of $[105, 151]$ is chosen)?
    
</div>

In [ ]:
from include.histogramm.HistOf import HistOf
import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams["figure.figsize"] = (12, 9)
h = HistOf(mc_dir="../data/for_long_analysis/mc_aftH",
           ru_dir="../data/for_long_analysis/ru_aftH", info=info)

In [ ]:
# example
h.variable("energy", 50, (0, 200))
ax = plt.gca()
ax.set_xlabel(r"$p_T$ in GeV")
ax.set_ylabel("Entries")
plt.show()

The uncertainties presented here are asymmetrical. For this purpose, a Poisson distribution (measured number of events) is determined from the expected value, the lower limit ( - 34%) and the upper limit (+34%), so that a 68% uncertainty interval can be specified. The asymmetry of the Poisson distribution is clearly visible for a small expected value. Only in the limit case of large expected value does the Poisson distribution transition to the Gaussian distribution and the uncertainties on a measured value become symmetrical.

In [ ]:
# other histograms

# Parameterization of the signal MC
--------------------------------------------------------

After choosing the best suited signal hypothesis, the distribution will be parameterized more precisely. This is done analogously to the background parameterization.

The motivation is analogous to that given in the section on background parameterization. Only the function forms used for parameterization change: These are bell-shaped in this case.

<div class="alert alert-info">

Parameterize the signal distribution with a suitable model. Also make sure that the model you choose is the most suitable one.

</div>

The students can choose from distributions that have already been implemented:
- Gauss distribution
- Cauchy distribution
- Voigt Profile
- Single - Sided - Crystal - Ball
- Double - Sided - Crystal - Ball


However, independently implemented distributions can also be used.

In [ ]:
from include.parameterization.SignalFunction import SignalFunction as SF

variant:

 1. <ins>The data from the histogram is raw output.</ins> 
<div class="alert alert-info">
     
    * Calculate the uncertainties on the individual entries of the unscaled histogram
    * Scale the histograms of the individual channels and combine them (and the calculated combined uncertainties) in a common histogram.

    (The individual parts can be accessed via the 'data' attribute, where it is a nested 'dict'. The first level contains the three individual channels. Each of the channels then contains the unscaled histogram entries, the correction factor and the data used for histogram creation).

</div>

In [ ]:
bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/"

In [ ]:
raw_mc_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir).get_mc_raw(tag="signal")
print(list(raw_mc_data.data.keys()))
raw_mc_data.data["mc_track_H_ZZ_4el_2012"]

In [ ]:
# parametrization 
import kafe2 as K2
# code

2. <ins>The data from the histogram is a summarized output</ins> 
    <div class="alert alert-info">
     
    Use the already combined values for a histogram fit.


</div>

In [ ]:
format_data = McFitInit(bins=bins, hist_range=hist_range, tag="background", mc_dir=mc_dir)
bins, hist_range, hist_entries, combined_hist_entries_errors = format_data.variables_for_hist_fit

The actual fit is done by using the histogram fit implemented in 'kafe2'. An example of a histogram fit can be found [here](https://kafe2.readthedocs.io/en/latest/parts/user_guide.html#example-5-histogram-fit)

It is advantageous to create a function or class to avoid lengthy repetitions.

In [ ]:
# parametrization
import kafe2 as K2
# code

# Determination of statistical significance
-------------------------

After successful parameterization of the background and the signal, the significance of the signal can be determined. The following parameterization is selected for this purpose:
$$ f(x, \alpha_\mathrm{S}) = \alpha_\mathrm{S} f_{\mathrm{S}}(x|\theta_{\mathrm{S}}) + (1 - \alpha_\mathrm{S}) f_{\mathrm{U}}(x|\theta_{\mathrm{U}}) \, .$$

The optimal parameter $\alpha_{\mathrm{S}}$ corresponds to the signal probability of this measurement. This determination is carried out with the help of the $-2\ln\mathcal{L(\alpha_{\mathrm{S}})}$ function which is to be implemented independently. The previously determined parameters $\theta_{\mathrm{S}}$ and $\theta_{\mathrm{U}}$ of the probability density function of the signal or background are fixed for the determination (profiled Likelihood).

The evaluation of the function value $-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)$ should be emphasized here. This corresponds to the assumption that there is no signal component in the existing measurement and the measurement can therefore be explained by the null hypothesis (only background). In asymptotic form, therefore, the significance can be determined analogously to the expression in the chapter on parameterization of the signal MC expression with: $$ Z = \sqrt{\frac{-2\ln\mathcal{L}(\alpha_{\mathrm{S}} = 0)}{\mathcal{L}_{\mathrm{min}}}} \, .$$

<div class="alert alert-info">
     
Implement the $-2\ln\mathcal{L(\alpha_S, m_{4\ell})}$ function (if not already present).Determine the signal probability plus uncertainty of the measurement.  Specify the significance with which a higgs-like boson is measured. In addition, vary the mass of the higgs-like boson for the two-dimensional variant and compare it with the literature. Visualize and discuss your results.
    
For the minimization you can use `minuit` as in QRT. The numerical integration can again be done using `scipy.integrate.quad`.
    
It is also possible to perform an `UninnedFit` using `kafe2` and implement the extended likelihood as a cost function independently.
</div>

In [ ]:
# measurement:
from include.histogramm.HistDataGetter import HistDataGetter as HDG

bins, hist_range, info = 999, (999.1, 999.2), [["2012"], ["B", "C"]]
mc_dir = "../data/for_long_analysis/mc_aftH/" 
ru_dir = "../data/for_long_analysis/ru_aftH/"
raw_data = HDG(bins=bins, hist_range=hist_range, 
           info=info, mc_dir=mc_dir, ru_dir=ru_dir).get_data_raw("mass_4l")
raw_data.data

In [ ]:
# code iminuit

def extended_nll_iminuit(alpha_s, mass=125.0):
    # code
    return mass

In [ ]:
# code kafe2

def extended_nll_kafe2(data, model, parameter_values, parameter_constraints):
    """
    
    :param data: measurement array
    :param model: model array (calculate individual in # code)
    :param parameter_values:
    :param parameter_constraints: (not used)
    :return:
    """
    
    # check if one of the parameter_values is nan
    if any(f"{par}" == "nan" for par in parameter_values):
        return np.inf
    
    # your code
    
    _total_log_likelihood = 0.0 # your code
    
    # check if _total_log_likelihood is nan
    if np.isnan(_total_log_likelihood):
        return np.inf
    return -2.0 * _total_log_likelihood

## 1D profiled Likelihood
-----------------------------------

The one-dimensional variant only varies the signal probability. The mass is held after the parameterization and is not varied anymore.

### Iminuit variant

In [ ]:
# code (iminuit minimization)

visualization:

In [ ]:
from include.stattest.PlotHelper import PlotHelper
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset

In [ ]:
alpha_ = np.linspace(0.0, 0.8, 1000)
nll_ = extended_nll(alpha_s=alpha_, mass=125)

# as example (to be replaced with actual nll array)
nll_ = 150*(alpha_ - 0.4) ** 2

z_ = np.sqrt(nll_[0])

# code to further customize the plot

fig, ax = plt.subplots(1,1)
PlotHelper.plot_nll_scan_main_window(ax_obj_=ax, 
                                     x_=alpha_, 
                                     y_=nll_, 
                                     z_=z_)

ax_in = inset_axes(ax, width='50%', height='40%', loc="upper right")
PlotHelper.plot_nll_scan_inside_window(ax_obj_=ax_in, 
                                       x_=alpha_, y_=nll_, 
                                       x_ticks_=PlotHelper.get_error_points_from_nll(x_=alpha_, nll_=nll_))
mark_inset(ax, ax_in, loc1=2, loc2=4, fc="none", ec="grey", lw=0.5, alpha=0.5)
plt.show()

### kafe2 variant

For the 'kafe2' variant, an 'uninned container' must be created and adjusted. For the 1D variant it is recommended to use the method `fix_parameter(par_name, value)`.
The significance can be calculated using the implemented `extended_nll_kafe2` cost function. It is recommended that you use the `limit_parameter` to restrict the parameter `alpha` to a reasonable interval.

In [ ]:
import kafe2 as K2

my_cost_function = K2.UnbinnedCostFunction_UserDefined(extended_nll_kafe2)

# more code

## 2D profiled Likelihood - Mass determination
-------------------

Finally, the mass is also determined. This is done by the two-dimensional variation - mass and signal probability.

### Iminuit variant

Similar to the 1D version, the best mass can be determined at the same time. The procedure is analogous to the 1D variant/QRT implementation.

In [ ]:
# code:
#     - minimization
#     - plots

### kafe2 variant

The only difference to the 1D variant is that the mass is no longer fixed by `fix_parameter`, but is varied in a certain interval (`limit_parameter`).

In [ ]:
# code

## Combination of the measurements: CMS and ATLAS
----------------------------

In order to obtain a better statistical significance, it is advantageous to include further measurements in the overall assessment. This can be done either by adding more data or by another experiment. Within the scope of this task the second option is performed.

For the ATLAS experiment, therefore, similar steps as above would have to be carried out, although many of the cuts show deviations. To shorten the work a bit, the final measurement is provided, as well as the histogram entries of the MC simulations.

In addition, the parameters of the most suitable functions (linear combination of centered legend trepolynomials up to the third order for the background and the DSCB distribution for the signal) are provided (but it is also possible to check these results yourself).

In [ ]:
import numpy as np
import pandas as pd
import kafe2 as K2
import os
from pprint import pprint

In [ ]:
atlas_data_dir_ = "../data/for_long_analysis/atlas_data_and_mc_hist/"
atlas_files = [os.path.join(atlas_data_dir_, file) for file in os.listdir(atlas_data_dir_) if ".csv" in file and "atlas" in file]
pprint(atlas_files)
hist_range_ = (106, 151)
num_bins = 45

data_A = np.loadtxt(atlas_files[2], delimiter=",")
data_A = data[(data >= hist_range_[0]) & (data <= hist_range_[1])]

mc_bac = np.loadtxt(atlas_files[1], delimiter=",").T
mc_bac_data = mc_bac[0]
mc_bac_error = mc_bac[1]

mc_sig = np.loadtxt(atlas_files[0], delimiter=",").T
mc_sig_data = mc_sig[0]
mc_sig_error = mc_sig[1]

In [ ]:
data_CMS = data
data_ATLAS = data_A

background_parameter_CMS = background_parameter
signal_parameter_CMS = signal_parameter

background_parameter_ATLAS  = np.array([0.025050083639971264, 
                                        0.00018744120591615378, 
                                        -1.178181555551107e-05, 
                                        1.1872474265844408e-07])
signal_parameter_ATLAS = np.array([2.1395802886491615, 
                                   124.19274632605674, 
                                   0.8324353801486924, 
                                   1.8034218165656613, 
                                   14.109501070824912, 
                                   19.9999999999894])
bfu_ATLAS = llc_background.grade_3
sfu_ATLAS = sf.DSCB

<div class="alert alert-info">
Combine the two measurements and determine the resulting mass of the Higgs boson, which is the signal probability and the significance of a measurement of the Higgs boson.

You can reuse/modify the functions you have already implemented.
    
Note:
    $$ \mathcal{L_{\mathrm{tot}}(\alpha_S, m_{4\ell})} = \mathcal{L_{1}(\alpha_S, m_{4\ell})} \mathcal{L_{2}(\alpha_S, m_{4\ell})} $$
</div>

In [ ]:
# code (1D and 2D minimization)